# [Happywhale - Whale and Dolphin Identification](https://www.kaggle.com/c/happy-whale-and-dolphin/overview)
## Identify whales and dolphins by unique characteristics

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/22962/logos/header.png?t=2021-03-17-22-44-0">

If you like this work, please upvote!

Please also this!

[Happywhale - 🐋 & 🐬 ID [EDA] | Kaggle](https://www.kaggle.com/tomato0813/happywhale-identification-eda)

[Happywhale - 🐋 & 🐬 ID [TensorFlow Tutorial] | Kaggle](https://www.kaggle.com/tomato0813/happywhale-id-tensorflow-tutorial)

## References for this notebook:

### Code
[1] [HappyWhale TFRecords | Kaggle](https://www.kaggle.com/ks2019/happywhale-tfrecords/data)

### Discussions
None

Please Upvote these work too!

### Others
[TFRecord and tf.train.Example &nbsp;|&nbsp; TensorFlow Core](https://www.tensorflow.org/tutorials/load_data/tfrecord#write_the_tfrecord_file)

Thanks guys!

In [ ]:
import os
import json
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import KFold
import tensorflow as tf
import IPython.display as display

In [ ]:
DATASET_NAME = f'happywhale-tfrecords-5743-v0'
FOLD = 1
N_FOLD = 5


train_filenams = [f'/tmp/{DATASET_NAME}/happywhale-train-{fold}.tfrecord' for fold in range(N_FOLD)]
test_filenames = [f'/tmp/{DATASET_NAME}/happywhale-test-{fold}.tfrecord' for fold in range(N_FOLD)]

In [ ]:
train_filenams

In [ ]:
test_filenames

In [ ]:
!rm -r /tmp/{DATASET_NAME}

os.makedirs(f'/tmp/{DATASET_NAME}', exist_ok=True)

with open('../input/kaggle-api-creds/kaggle.json') as f:
    kaggle_creds = json.load(f)
    
os.environ['KAGGLE_USERNAME'] = kaggle_creds['username']
os.environ['KAGGLE_KEY'] = kaggle_creds['key']

!kaggle datasets init -p /tmp/{DATASET_NAME}


with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json') as f:
    dataset_meta = json.load(f)
dataset_meta['id'] = f'tomato0813/{DATASET_NAME}'
dataset_meta['title'] = DATASET_NAME
with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json', "w") as outfile:
    json.dump(dataset_meta, outfile)
print(dataset_meta)

!ls /tmp/{DATASET_NAME}

!kaggle datasets create -u -p /tmp/{DATASET_NAME} 

In [ ]:
train_df = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
train_df.head()

In [ ]:
train_df['image_path'] = ['../input/happy-whale-and-dolphin/train_images/' + img for img in train_df['image']]
train_df.head()

In [ ]:
id_to_label = dict((i_id, index) for index, i_id in enumerate(train_df['individual_id'].unique()))

In [ ]:
labels = [id_to_label[i_id] for i_id in train_df['individual_id']]
labels[:20]

In [ ]:
train_df['label'] = labels
train_df.head(20)

In [ ]:
train_df['fold'] = None

kf = KFold(n_splits=N_FOLD)
for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    train_df['fold'].iloc[test_index] = i

train_df.head()

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def image_example(image_string, label):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
        'image_raw': _bytes_feature(image_string),
        'label': _int64_feature(label),
    }

    # Create a Features message using tf.train.Example.
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
def create_train_tf_record(fold):
    record_file = train_filenams[fold]
    df = train_df[train_df.fold == fold]
    with tf.io.TFRecordWriter(record_file) as writer:
        for i, row in df.iterrows():
            image_string = tf.io.read_file(row.image_path)
            label = row.label
            tf_example = image_example(image_string, label)
            writer.write(tf_example.SerializeToString())

In [ ]:
import joblib
_ = joblib.Parallel(n_jobs=8)(
        joblib.delayed(create_train_tf_record)(fold) for fold in tqdm(range(N_FOLD))
    )

In [ ]:
test_df = pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv')
test_df.head()

In [ ]:
test_df['image_path'] = ['../input/happy-whale-and-dolphin/test_images/' + img for img in test_df['image']]
test_df.head()

In [ ]:
test_df['fold'] = None

kf = KFold(n_splits=N_FOLD)
for i, (train_index, test_index) in enumerate(kf.split(test_df)):
    test_df['fold'].iloc[test_index] = i

test_df.head()

In [ ]:
def create_test_tf_record(fold):
    record_file = test_filenames[fold]
    df = test_df[test_df.fold == fold]
    with tf.io.TFRecordWriter(record_file) as writer:
        for i, row in df.iterrows():
            image_string = tf.io.read_file(row.image_path)
            label = -1
            tf_example = image_example(image_string, label)
            writer.write(tf_example.SerializeToString())

In [ ]:
import joblib
_ = joblib.Parallel(n_jobs=8)(
        joblib.delayed(create_test_tf_record)(fold) for fold in tqdm(range(N_FOLD))
    )

In [ ]:
from datetime import datetime
version_name = datetime.now().strftime("%Y%m%d-%H%M%S")
print(version_name)

In [ ]:
!kaggle datasets version -m {version_name} -p /tmp/{DATASET_NAME} -r zip -q

In [ ]:
raw_image_dataset = tf.data.TFRecordDataset(test_filenames[FOLD])
raw_image_dataset

In [ ]:
# Create a description of the features.
image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_image_function(example_proto):
    # Parse the input `tf.train.Example` proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, image_feature_description)

In [ ]:
parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset